<a href="https://colab.research.google.com/github/lweislo/UCI_data/blob/main/UCI_Calendar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This script will get the full calendar from the UCI API.


In [ ]:
# Dependencies
import pandas as pd
import requests
import json
from bs4 import BeautifulSoup
from google.colab import files
# First get the URLs for each race since the API doesn't have category, class or country
def get_calendar_links(url):
    data = requests.get(url).json()
    data = data['items']
    event_list = []
    try:
        for item in data:
            for i in item['items']:
                for race in i['items']:
                    event_list.append("https://www.uci.org" + race['detailsLink']['url'])
    except TypeError:
        print(f'Missing data from an item, skipping')
        pass
    get_race_calendar_data(event_list)

# Then iterate through all the URLs to get the information you need
def get_race_calendar_data(url_list):
    df = pd.DataFrame()
    for item in url_list:
        page = requests.get(item)
        print(f'Getting data from {item}')
        try:
            if page.status_code == 200:
                content = page.content
                soup = BeautifulSoup(content, "html5lib")
                info = soup.find('div',{'data-component':'CompetitionDetailsModule'})
                race_info = json.loads(info['data-props'])
                race_dict = {'Race': race_info['competitionDetails']['name'], 'Class':race_info['competitionDetails']['competitionClass'], 'Dates': race_info['competitionDetails']['dates'], 'Country':race_info['competitionDetails']['country'],'Category':race_info['schedule']['items'][0]['races'][0]['category']}
                df = df.append(race_dict, ignore_index=True)
        except TyperError:
            print(f'Missing data from {item} - skipped')
            pass
        
    df_deduped = df.drop_duplicates(subset=['Race','Category'])
    outfile = 'uci_calendar.txt'
    with open(outfile, 'w') as file:
        file.write(df_deduped.to_csv(header=True, index=False, encoding='UTF-8', sep='\t'))
    files.download(outfile)

In [ ]:
# Enter the URL for the API - this is for road. Change last three letters to MTB, CRO, PIS, BMX, BFR for mountain bike, cross, track, BMX, freestyle
url = 'https://www.uci.org/api/calendar/upcoming?discipline=ROA'


In [ ]:
# Run the scraper
get_calendar_links(url)